In [1]:
import torch

In [2]:
from sylvestorch.models import SylvesterNet

In [3]:
sylvester = SylvesterNet(3, 10, 10)

In [4]:
x = torch.randn((4, 10))
y = sylvester(x)

In [5]:
y

(tensor([[ 0.2177,  0.1690,  0.1587,  2.1493,  1.8501,  0.1951,  0.9745, -0.0483,
          -0.7731,  0.6554],
         [-2.2199, -0.8726, -0.7277,  3.3121, -0.6518, -0.4210, -0.6650, -0.2175,
           0.5399, -0.4393],
         [-1.3599, -1.1895,  1.3351,  0.3362,  0.2412, -1.6154, -2.1149,  0.7816,
          -2.2940, -1.0264],
         [-0.3664,  0.7470, -0.4199,  1.7447,  0.2875,  1.1277, -3.8760, -0.9504,
          -1.6154,  0.1789]], grad_fn=<AddBackward0>),
 tensor([ 0.0341,  0.0178, -0.0102, -0.0136], grad_fn=<AddBackward0>))